# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
file = "../output_data/cities.csv"

cities = pd.read_csv(file)
cities.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Store latitude and longitude in locations
locations = cities[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = cities["Humidity"]

In [11]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [105]:
#Create a Dataframe that narrows down to ideal weather conditions for vacation to these:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.


#A max temperature lower than 80 degrees but higher than 70.
ideal_weather1 = cities.loc[cities["Max Temp"]<80,:]
#ideal_weather1

ideal_weather2 = ideal_weather1.loc[cities["Max Temp"]>70,:]
#ideal_weather2

#Wind speed less than 10 mph.
ideal_weather3 = ideal_weather2.loc[ideal_weather2["Wind Speed"] <10,:]
#ideal_weather3

#Zero cloudiness.
ideal_weather4 = ideal_weather3.loc[ideal_weather3["Cloudiness"] == 0]
ideal_weather4


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [111]:
hotel_df = pd.DataFrame()
ideal_weather5 = pd.DataFrame(ideal_weather4)
hotel_df['City'] = ideal_weather5['City']
hotel_df['Country'] = ideal_weather5['Country']
hotel_df['Latitude'] = ideal_weather5['Lat']
hotel_df['Longitude'] = ideal_weather5['Lng']
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
7,opuwo,NaN,-18.06,13.84,
70,ponta do sol,BR,-20.63,-46.00,
88,cidreira,BR,-30.17,-50.22,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,


In [154]:
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get hotel type from df
    Country = row['Country']
    City = row['City']
    Latitude = row['Latitude']
    Longitude = row['Longitude']
   

    # add keyword to params dict
    #params['keyword'] = hotel

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {City}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={Latitude},{Longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][5]['name']
        print(f"Closest hotel to  {City} at {Latitude},{Longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")




Retrieving Results for Index 7: opuwo.
Closest hotel to  opuwo at -18.06,13.84 is Dunns Opuwo.
------------
Retrieving Results for Index 70: ponta do sol.
Closest hotel to  ponta do sol at -20.63,-46.0 is Casa Azul Suítes.
------------
Retrieving Results for Index 88: cidreira.
Closest hotel to  cidreira at -30.17,-50.22 is Pond Country Club.
------------
Retrieving Results for Index 138: morondava.
Closest hotel to  morondava at -20.3,44.28 is FJKM Mahagaga Morondava.
------------
Retrieving Results for Index 139: vaini.
Closest hotel to  vaini at 15.34,74.49 is Shri Ram Temple Shindoli.
------------
Retrieving Results for Index 176: nador.
Closest hotel to  nador at 35.17,-2.93 is Café Titanic مقهى تايتانيك.
------------
Retrieving Results for Index 297: mogok.
Closest hotel to  mogok at 22.92,96.51 is Badamyar Nga Mauk Guest House or Royal Ruby Guest House - Burmese Only.
------------
Retrieving Results for Index 444: birjand.
Closest hotel to  birjand at 32.86,59.22 is بیمه معلم mo

In [155]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
7,opuwo,NaN,-18.06,13.84,Dunns Opuwo
70,ponta do sol,BR,-20.63,-46.00,Casa Azul Suítes
88,cidreira,BR,-30.17,-50.22,Pond Country Club
138,morondava,MG,-20.30,44.28,FJKM Mahagaga Morondava
139,vaini,IN,15.34,74.49,Shri Ram Temple Shindoli
176,nador,MA,35.17,-2.93,Café Titanic مقهى تايتانيك
297,mogok,MM,22.92,96.51,Badamyar Nga Mauk Guest House or Royal Ruby Gu...
444,birjand,IR,32.86,59.22,بیمه معلم moallem insurance


In [156]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [157]:
# Add marker layer ontop of heat map
# find the closest restaurant of each type to coordinates

